# Import dataset

In [1]:
import pandas as pd

In [3]:
import pandas as pd
import json
from functools import reduce
import numpy as np
df = pd.read_csv('/content/911_Calls_for_Service_(Last_30_Days).csv')
df = df.dropna(subset=['zip_code', 'neighborhood'])
data = df.to_dict('records')

# Bagian 1 Model Populasi Polisi Detroit

In [4]:

list_dict_output = [x for x in data if x["zip_code"] and x["neighborhood"] and x["totalresponsetime"] and x["time_on_scene"] and x["totaltime"]]

total_response_time = sum(x.get("totalresponsetime", 0) for x in list_dict_output) / len(list_dict_output)
delivery_time = sum(int(x.get("time_on_scene", 0)) if not np.isnan(x.get("time_on_scene", 0)) else 0 for x in list_dict_output) / len(list_dict_output)
total_time = sum(int(x.get("totaltime", 0)) if not np.isnan(x.get("totaltime", 0)) else 0 for x in list_dict_output) / len(list_dict_output)

print("Total Response Time Average: ", total_response_time)
print("Delivery Time Average: ", delivery_time)
print("Total Time Average: ", total_time)

Total Response Time Average:  nan
Delivery Time Average:  32.57122807017544
Total Time Average:  55.91649122807018


# Modelkan Neighborhood Samples

In [5]:
neighborhoods = list(set(map(lambda x: x["neighborhood"], data)))
dictNeighbor = {}
for neighborhood in neighborhoods:
    dictNeighbor[neighborhood] = list(filter(lambda x: x["neighborhood"] == neighborhood, data))

for neighborhood in dictNeighbor:
    dictNeighbor[neighborhood] = {
        "total_response_time": reduce(lambda x, y: x + y,map(lambda x: x.get("totalresponsetime", 0),dictNeighbor[neighborhood]))
        / len(dictNeighbor[neighborhood]),
        "delivery_time": reduce(lambda x, y: x + y,map(lambda x: int(x.get("time_on_scene", 0)) if not np.isnan(x.get("time_on_scene", 0)) else 0,dictNeighbor[neighborhood]))
        / len(dictNeighbor[neighborhood]),
        "total_time": reduce(lambda x, y: x + y,map(lambda x: int(x.get("totaltime", 0)) if not np.isnan(x.get("totaltime", 0)) else 0,dictNeighbor[neighborhood]))
        / len(dictNeighbor[neighborhood]),
    }

dictNeighbor["All Detroit"] = {
    "total_response_time": total_response_time,
    "delivery_time": delivery_time,
    "total_time": total_time,
}

dictNeighbor["All Detroit"]

{'total_response_time': nan,
 'delivery_time': 32.57122807017544,
 'total_time': 55.91649122807018}

# Output JSON

In [7]:

with open("DataPopulasiDetroit.json", "w") as outfile:
    json.dump(list(dictNeighbor.values()), outfile)